# Pattern Searching Algorithms Practice

## Rabin-Karp Algorithm

In [ ]:
# Implementation of Rabin-Karp Algorithm
# to search a pattern in a given text

# Number of characters in the input alphabet
d = 256

# Function to search for pattern occurrences
def search(pattern, text, prime):
    # Get the length of pattern and text
    M = len(pattern)
    N = len(text)

    # Initialize hash values for pattern and text
    pattern_hash = 0  # Hash value for the pattern
    text_hash = 0     # Hash value for the current window of the text
    h = 1  # The multiplier used to maintain the rolling hash value

    # Precompute (d^(M-1)) % prime to use in rolling hash calculation
    for i in range(M - 1):
        h = (h * d) % prime

    # Calculate the initial hash values for the pattern and the first window of the text
    for i in range(M):
        pattern_hash = (d * pattern_hash + ord(pattern[i])) % prime
        text_hash = (d * text_hash + ord(text[i])) % prime

    # Slide the pattern over the text one character at a time
    for i in range(N - M + 1):
        # If the hash values match, check the characters one by one
        if pattern_hash == text_hash:
            # Verify each character to avoid hash collision
            match = True
            for j in range(M):
                if text[i + j] != pattern[j]:
                    match = False
                    break

            # If all characters match, print the starting index of the match
            if match:
                print(f"Pattern found at index {i}")

        # Calculate the hash value for the next window of text
        # by removing the leading character and adding a trailing character
        if i < N - M:
            text_hash = (d * (text_hash - ord(text[i]) * h) + ord(text[i + M])) % prime

            # If the hash value becomes negative, convert it to a positive value
            if text_hash < 0:
                text_hash = text_hash + prime

# Driver code to test the implementation
if __name__ == "__main__":
    text = "GOOSES AND GOOSES"
    pattern = "GOOSE"

    # A prime number used for hashing
    prime = 101

    # Call the search function
    search(pattern, text, prime)

Pattern found at index 0
Pattern found at index 11


## KMP Algorithm

In [1]:
def computeLPSArray(pattern):
    # Initialize variables for LPS computation
    length = 0  # Length of the previous longest prefix suffix
    lps = [0] * len(pattern)
    i = 1

    # Compute the LPS array
    while i < len(pattern):
        if pattern[i] == pattern[length]:
            length += 1
            lps[i] = length
            i += 1
        else:
            if length != 0:
                length = lps[length - 1]
            else:
                lps[i] = 0
                i += 1
    return lps


def KMPSearch(pattern, text):
    # Precompute the LPS array for the pattern
    lps = computeLPSArray(pattern)
    result = []  # Store positions of matches

    i = 0  # Index for text
    j = 0  # Index for pattern

    # Search for pattern in text
    while (len(text) - i) >= (len(pattern) - j):
        if pattern[j] == text[i]:
            j += 1
            i += 1

        if j == len(pattern):
            result.append(i - j + 1)  # Store the 1-based index of the match
            j = lps[j - 1]
        elif i < len(text) and pattern[j] != text[i]:
            if j != 0:
                j = lps[j - 1]
            else:
                i += 1
    return result


# Example usage
text = "gooseforgoose"
pattern = "goose"
result = KMPSearch(pattern, text)

# Print all the occurrences (1-based indices)
for index in result:
    print(index, end=' ')

1 9 

## Z algorithm

In [2]:
# Python3 program to implement the Z algorithm
# for pattern searching, with modified comments and code refactoring.

def calculate_z_array(string, z):
    """
    Fills the Z array for the given string, where Z[i] represents the length of the longest substring
    starting from 'i' which is also a prefix of 'string'.
    """
    n = len(string)
    left, right = 0, 0

    for i in range(1, n):
        # If i is outside the current [left, right] window, calculate Z[i] from scratch.
        if i > right:
            left, right = i, i
            while right < n and string[right - left] == string[right]:
                right += 1
            z[i] = right - left
            right -= 1
        else:
            # Use previously computed values to optimize Z[i]
            k = i - left
            if z[k] < right - i + 1:
                z[i] = z[k]
            else:
                left = i
                while right < n and string[right - left] == string[right]:
                    right += 1
                z[i] = right - left
                right -= 1

def z_algorithm_search(text, pattern):
    """
    Searches for all occurrences of 'pattern' in 'text' using the Z algorithm.
    """
    # Create a concatenated string "pattern$text"
    concatenated = pattern + "$" + text
    concatenated_length = len(concatenated)

    # Construct the Z array for the concatenated string
    z = [0] * concatenated_length
    calculate_z_array(concatenated, z)

    # Traverse the Z array to find the pattern matches
    for i in range(concatenated_length):
        if z[i] == len(pattern):
            print(f"Pattern found at index {i - len(pattern) - 1}")

# Driver Code
if __name__ == "__main__":
    text = "GOOSES AND GOOSES"
    pattern = "GOOSES"
    z_algorithm_search(text, pattern)


Pattern found at index 0
Pattern found at index 11


## kasai’s Algorithm

In [3]:
class Suffix:
    def __init__(self):
        # Each suffix has an index representing its starting position
        # and a rank array to help with sorting suffixes
        self.index = 0
        self.rank = [0, 0]

# Function to build the suffix array
# txt: input string
# n: length of the input string
def buildSuffixArray(txt, n):
    # Create an array of Suffix objects to hold information for each suffix
    suffixes = [Suffix() for _ in range(n)]

    # Initialize suffixes with their original ranks based on the first two characters
    for i in range(n):
        suffixes[i].index = i
        suffixes[i].rank[0] = ord(txt[i]) - ord('a')  # Rank based on the first character
        suffixes[i].rank[1] = ord(txt[i + 1]) - ord('a') if i + 1 < n else -1  # Rank based on the second character

    # Sort suffixes based on the first two characters
    suffixes.sort(key=lambda x: (x.rank[0], x.rank[1]))

    # Temporary array to hold updated indices of suffixes
    ind = [0] * n

    # Loop to update ranks of suffixes and sort them accordingly
    k = 4  # Initial length of substrings to consider
    while k < 2 * n:
        # Assign new ranks to suffixes based on their previous ranks
        rank = 0
        prev_rank = suffixes[0].rank[0]
        suffixes[0].rank[0] = rank
        ind[suffixes[0].index] = 0

        # Assign ranks to suffixes
        for i in range(1, n):
            if suffixes[i].rank[0] == prev_rank and suffixes[i].rank[1] == suffixes[i - 1].rank[1]:
                prev_rank = suffixes[i].rank[0]
                suffixes[i].rank[0] = rank
            else:
                prev_rank = suffixes[i].rank[0]
                rank += 1
                suffixes[i].rank[0] = rank
            ind[suffixes[i].index] = i

        # Update the second rank for each suffix
        for i in range(n):
            nextindex = suffixes[i].index + k // 2
            suffixes[i].rank[1] = suffixes[ind[nextindex]].rank[0] if nextindex < n else -1

        # Sort suffixes based on the new ranks
        suffixes.sort(key=lambda x: (x.rank[0], x.rank[1]))

        # Double the value of k to consider longer substrings in each iteration
        k *= 2

    # Extract and return the suffix array (which contains the starting indices of sorted suffixes)
    suffixArr = [suffix.index for suffix in suffixes]
    return suffixArr

# Function to build the LCP array using Kasai's algorithm
def kasai(txt, suffixArr):
    n = len(suffixArr)
    lcp = [0] * n  # Initialize LCP array
    invSuff = [0] * n  # Array to store inverse of suffix array indices

    # Fill the inverse suffix array
    for i in range(n):
        invSuff[suffixArr[i]] = i

    k = 0  # Length of the longest common prefix

    # Build the LCP array
    for i in range(n):
        # If this suffix is the last suffix, there is no next suffix to compare it to
        if invSuff[i] == n - 1:
            k = 0
            continue

        # Get the index of the next suffix to compare
        j = suffixArr[invSuff[i] + 1]

        # Compare characters of the current suffix and the next suffix
        while i + k < n and j + k < n and txt[i + k] == txt[j + k]:
            k += 1

        # Store the length of the longest common prefix for this suffix
        lcp[invSuff[i]] = k

        # Reduce k for the next iteration (since the next LCP is at least k-1)
        if k > 0:
            k -= 1

    return lcp

# Utility function to print an array
def printArr(arr):
    print(" ".join(map(str, arr)))

# Driver program to test the above functions
if __name__ == "__main__":
    input_str = "banana"

    # Build suffix array for the input string
    suffixArr = buildSuffixArray(input_str, len(input_str))
    print("Suffix Array:")
    printArr(suffixArr)

    # Build LCP array for the input string
    lcp = kasai(input_str, suffixArr)
    print("\nLCP Array:")
    printArr(lcp)

Suffix Array:
5 3 1 0 4 2

LCP Array:
1 3 0 0 2 0


## Aho-Corasick Algorithm

In [4]:
from collections import defaultdict

# For simplicity, Arrays and Queues have been implemented using lists.
# For better performance, consider using a more efficient data structure.
class AhoCorasick:
    def __init__(self, words):
        # Maximum number of states in the matching machine.
        # This should be equal to the sum of the lengths of all keywords.
        self.max_states = sum([len(word) for word in words])

        # Maximum number of characters (for alphabets [a-z]).
        self.max_characters = 26

        # Output function, implemented using 'out[]'.
        # Each bit 'i' in this mask is 1 if the word with index 'i' appears when the machine enters this state.
        self.out = [0] * (self.max_states + 1)

        # Failure function, implemented using 'fail[]'.
        # Contains the fail state value for each state. Initialized to -1.
        self.fail = [-1] * (self.max_states + 1)

        # Goto function (or Trie), implemented using 'goto[][]'.
        # Number of rows = max_states + 1, number of columns = max_characters (26).
        self.goto = [[-1] * self.max_characters for _ in range(self.max_states + 1)]

        # Convert all words to lowercase to make the search case insensitive.
        for i in range(len(words)):
            words[i] = words[i].lower()

        # List of words that will be used to create the Trie.
        # The index of each keyword is important: "out[state] & (1 << i)" is > 0 if we just found word[i] in the text.
        self.words = words

        # Once the Trie has been built, it will contain the number of nodes in Trie which is the total number of states required.
        self.states_count = self.__build_matching_machine()

    # Builds the string matching machine (Trie).
    # Returns the number of states that the built machine has.
    # States are numbered from 0 up to the return value - 1, inclusive.
    def __build_matching_machine(self):
        k = len(self.words)

        # Initially, we only have the root state (0).
        states = 1

        # Building the Trie by populating the 'goto' function.
        for i in range(k):
            word = self.words[i]
            current_state = 0

            # Process each character of the current word.
            for character in word:
                ch = ord(character) - 97  # ASCII value of 'a' is 97.

                # Allocate a new node (create a new state) if it doesn't exist.
                if self.goto[current_state][ch] == -1:
                    self.goto[current_state][ch] = states
                    states += 1

                current_state = self.goto[current_state][ch]

            # Add the current word to the output function.
            self.out[current_state] |= (1 << i)

        # For all characters that don't have an edge from the root in the Trie,
        # add a goto edge to the root itself.
        for ch in range(self.max_characters):
            if self.goto[0][ch] == -1:
                self.goto[0][ch] = 0

        # Failure function is computed using breadth-first search (BFS).
        queue = []

        # Iterate over all possible inputs.
        for ch in range(self.max_characters):

            # All nodes of depth 1 have failure function value as 0.
            if self.goto[0][ch] != 0:
                self.fail[self.goto[0][ch]] = 0
                queue.append(self.goto[0][ch])

        # Process the rest of the nodes using BFS.
        while queue:
            # Remove the front state from the queue.
            state = queue.pop(0)

            # For each character, compute the failure function.
            for ch in range(self.max_characters):

                # If the 'goto' function is defined for character 'ch'.
                if self.goto[state][ch] != -1:
                    failure = self.fail[state]

                    # Find the deepest node labeled by a proper suffix of the string from root to the current state.
                    while self.goto[failure][ch] == -1:
                        failure = self.fail[failure]

                    failure = self.goto[failure][ch]
                    self.fail[self.goto[state][ch]] = failure

                    # Merge output values.
                    self.out[self.goto[state][ch]] |= self.out[failure]

                    # Add the next level node to the queue.
                    queue.append(self.goto[state][ch])

        return states

    # Returns the next state of the machine using 'goto' and 'failure' functions.
    # current_state - The current state of the machine. Must be between 0 and the number of states - 1, inclusive.
    # next_input - The next character that enters into the machine.
    def __find_next_state(self, current_state, next_input):
        answer = current_state
        ch = ord(next_input) - 97  # ASCII value of 'a' is 97.

        # If 'goto' is not defined, use the failure function.
        while self.goto[answer][ch] == -1:
            answer = self.fail[answer]

        return self.goto[answer][ch]

    # This function finds all occurrences of all words in the given text.
    def search_words(self, text):
        # Convert the text to lowercase to make the search case insensitive.
        text = text.lower()

        # Initialize current state to 0 (root).
        current_state = 0

        # A dictionary to store the result.
        # Key is the found word, value is a list of all occurrence start indices.
        result = defaultdict(list)

        # Traverse the text through the built machine to find all occurrences of words.
        for i in range(len(text)):
            current_state = self.__find_next_state(current_state, text[i])

            # If no match is found, continue to the next character.
            if self.out[current_state] == 0:
                continue

            # If a match is found, store the word in the result dictionary.
            for j in range(len(self.words)):
                if (self.out[current_state] & (1 << j)) > 0:
                    word = self.words[j]
                    # Start index of the word is (i - len(word) + 1).
                    result[word].append(i - len(word) + 1)

        # Return the final result dictionary.
        return result


# Driver code
if __name__ == "__main__":
    words = ["he", "she", "hers", "his"]
    text = "ahishers"

    # Create an object to initialize the Trie.
    aho_corasick = AhoCorasick(words)

    # Get the result.
    result = aho_corasick.search_words(text)

    # Print the result.
    for word in result:
        for i in result[word]:
            print("Word", word, "appears from", i, "to", i + len(word) - 1)

Word his appears from 1 to 3
Word he appears from 4 to 5
Word she appears from 3 to 5
Word hers appears from 4 to 7


## Finite Automata algorithm

In [5]:
# Python program to implement Finite Automata
# for pattern searching algorithm

NO_OF_CHARS = 256

def getNextState(pat, M, state, x):
    '''
    Calculate the next state for the finite automaton.
    pat: Pattern for which we are creating the automaton
    M: Length of the pattern
    state: Current state in the automaton
    x: Current character to process (as an ASCII value)
    '''
    # If the character matches the next character in the pattern,
    # increment the state by one
    if state < M and x == ord(pat[state]):
        return state + 1

    # Iterate from the largest possible prefix to the smallest to find the next state
    for ns in range(state, 0, -1):
        if ord(pat[ns - 1]) == x:
            for i in range(ns - 1):
                if pat[i] != pat[state - ns + 1 + i]:
                    break
            else:
                return ns
    return 0

def computeTF(pat, M):
    '''
    Build the transition function (TF) table which represents the finite automaton for the given pattern.
    pat: Pattern for which the automaton is being built
    M: Length of the pattern
    '''
    global NO_OF_CHARS

    # Initialize the transition function table with zeros
    TF = [[0 for _ in range(NO_OF_CHARS)] for _ in range(M + 1)]

    # Populate the transition function table
    for state in range(M + 1):
        for x in range(NO_OF_CHARS):
            TF[state][x] = getNextState(pat, M, state, x)

    return TF

def search(pat, txt):
    '''
    Search for all occurrences of the pattern in the given text using the finite automaton.
    pat: The pattern to search for
    txt: The text in which to search
    '''
    global NO_OF_CHARS
    M = len(pat)
    N = len(txt)

    # Build the transition function table for the given pattern
    TF = computeTF(pat, M)

    # Process the text using the finite automaton
    state = 0
    for i in range(N):
        state = TF[state][ord(txt[i])]
        if state == M:
            print("Pattern found at index: {}".format(i - M + 1))

# Driver function to test the above implementation
if __name__ == '__main__':
    txt = "AABAACAADAABAAABAA"
    pat = "AABA"
    search(pat, txt)

Pattern found at index: 0
Pattern found at index: 9
Pattern found at index: 13
